# Analyse AIRSS Search results

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
#from toolchest.airssutils import read_res, read_res_title, read_stream
import pandas as pd
import numpy as np

Read in the data using cryan

## Load VASP resulst computed from AiiDA

In [2]:
%aiida
from aiida.orm import Group, QueryBuilder, StructureData,Code, Computer
# Gruop containing the res structures for easy access
res_structgroup = Group.objects.get_or_create('lifepo4-airss-refine-res-structure')[0]
vasp_relax = WorkflowFactory('vaspu.relax')
mp_struct_group = Group.objects.get(label='lifepo4-mp-refine')

In [3]:
import re
# Patter matching the name
pat = re.compile(r'([\w\-_]+)\.res')

def get_functional(incar, pot):
    """
    Return the name of the functional
    """
    if 'metagga' in incar:
        return incar['metagga']
    if pot.startswith('LDA'):
        if incar.get('gga'):
            return 'gga+ldapp'
        else:
            return 'lda'
    elif pot.startswith('PBE'):
        gga = incar.get('gga')
        hf = incar.get('lhfcalc')
        if not hf:
            if (not gga) or gga.lower() == 'pe':
                return 'pbe'
            if gga.lower() == 'ps':
                return 'pbesol'
        else:
            if (not gga) or gga.lower() == 'pe':
                if incar.get('aexx') in [0.25, None] and (incar.get('hfscreen') - 0.2 < 0.01):
                    return 'hse06'
            
    return 'unknown'

def get_dataframe(group, lkey=0):
    """Obtained relaxed structures for a particular group"""
    q = QueryBuilder()
    q.append(Group, filters={'id': group.pk})
    q.append(StructureData, with_group=Group, project=['id'])
    q.append(vasp_relax, with_incoming=StructureData, filters={'attributes.exit_status': {'in': [0, 406, 600]}},
             project=['uuid', 'label'])
    q.append(StructureData, with_incoming=vasp_relax, project=['id', '*'], edge_filters={'label': 'relax__structure'})
    q.append(Dict, with_outgoing=vasp_relax, filters={
        'or': [{'attributes.vasp.encut': 520}, {'attributes.incar.encut': 520}]},
             edge_filters={'label': {'like': '%parameters'}},
            project=['attributes'])
    q.append(Dict, with_incoming=vasp_relax, project=['attributes.total_energies'], edge_filters={'label': 'misc'})
    q.distinct()
    results = []
    for spk, rpk, rlabel, rspk, struct, inputd, eng  in q.all():
        funits = struct.get_pymatgen().composition.get_integer_formula_and_factor()[1]
        name = rlabel.split()[lkey]
        eng = list(eng.values())[0]
        if 'incar' in inputd:
            inputd = inputd['incar']
        else:
            inputd = inputd['vasp']
        results.append({'relax_label':rlabel, 
                        'functional': get_functional(inputd, 'PBE.54'),
                        'label': str(name), 'relax_work': rpk, 'relaxed': rspk, 
                        'energy': eng, 'energy_fu': eng / funits, 'nform': funits,
                        'volume': struct.get_cell_volume(), 'volume_fu': struct.get_cell_volume() / funits})
    return pd.DataFrame.from_records(results)


In [4]:
res_vasp_df = get_dataframe(res_structgroup)
mp_vasp_df = get_dataframe(mp_struct_group, lkey=1)

vasp_df = pd.concat([res_vasp_df, mp_vasp_df], axis=0)
vasp_df['label'] = vasp_df['label'].str.replace('.res', '')

for funct in ['pbe', 'pbesol']:
    ref = vasp_df.loc[(vasp_df['label'] == "2LFP_REF-mp-19017")&(vasp_df.functional == funct)]
    vasp_df.loc[vasp_df.functional == funct, 'e_vasp_ref'] = vasp_df.loc[vasp_df.functional == funct, 'energy_fu'] - ref.energy_fu.iloc[0]

tagf= lambda x: 'MP' if 'mp' in x else "AIRSS"
vasp_df['type'] = [ tagf(x) for x in vasp_df.label ]
vasp_df_mp = vasp_df[vasp_df.type == 'MP']
vasp_df_airss = vasp_df[vasp_df.type == 'AIRSS']

vasp_df.loc[vasp_df['label'].str.contains('9153-6'), 'type'] = 'MP'


vasp_df['natom'] = vasp_df['nform'] * 7
vasp_df['e_vasp_pa'] = vasp_df['e_vasp_ref'] / 7

In [5]:
vasp_pbe = vasp_df[vasp_df.functional == 'pbe']
vasp_pbesol = vasp_df[vasp_df.functional == 'pbesol']


# Export the data as plain CSV

In [7]:
DATADIR = '../CoM_paper/data/'
vasp_pbe.to_csv(DATADIR + 'lifepo4-pbe-data.csv')
vasp_pbesol.to_csv(DATADIR + 'lifepo4-pbesol-data.csv')

Export relaxated structures